In [2]:
import transformers
import inseq
import torch
import pandas as pd

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
transcripts = pd.read_csv('/home/ada/humor/humor/standup_transcripts.csv')

In [4]:
transcripts.head()

,comedian,transcript
0,Anthony_Jeselnik,"When I was a kid, I used to fantasize about ge..."
1,Anthony_Jeselnik_2,No one should ever ask me to speak at anyone’...
2,Ali_Wong,"The last time I was at home in San Francisco, ..."
3,Ali_Wong_2,If you really want to piss off a really hot gi...
4,Chelsea_Peretti,A friend of a friend just posted like 500 enga...


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import nltk.data

def split_transcript(transcript):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(transcript.strip())
    total_sentences = len(sentences)
    sentences_per_part = total_sentences // 4

    part1 = sentences[:sentences_per_part]
    part2 = sentences[sentences_per_part:2*sentences_per_part]
    part3 = sentences[2*sentences_per_part:3*sentences_per_part]
    part4 = sentences[3*sentences_per_part:]

    part1_text = ' '.join(part1)
    part2_text = ' '.join(part2)
    part3_text = ' '.join(part3)
    part4_text = ' '.join(part4)

    return [part1_text, part2_text, part3_text, part4_text]


In [7]:
transcript_ali_wong = transcripts.loc[transcripts['comedian'] == 'Ali_Wong', 'transcript'].values[0]
parts = split_transcript(transcript_ali_wong)

In [9]:
Q8BIT = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True,
    llm_int8_has_fp16_weight=True,
)

model = inseq.load_model(
    model="google/gemma-2b-it", 
    attribution_method="integrated_gradients",
    model_kwargs=dict(
        device_map="cuda:0",
     #   torch_dtype=torch.bfloat16,
        trust_remote_code=False,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
        # quantization_config=Q8BIT,
    )
)

print("memory cost: ", model.model.get_memory_footprint() >> 20, "Mb")

/home/ada/humor/.venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.32s/it]


memory cost:  9560 Mb


In [12]:
for part in parts:
    prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

    out = model.attribute(
    prompt,
    n_steps=16,
    batch_size=1,
    device=model.device,
    generation_args=dict(
        max_new_tokens=65,
    )
    )

    out.show()

Attributing with integrated_gradients...:  90%|████████▉ | 107/119 [00:46<00:17,  1.46s/it]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.63 GiB (GPU 0; 23.58 GiB total capacity; 21.69 GiB already allocated; 1.46 GiB free; 21.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF